# チュートリアル
これは"タダノ クレーン旋回操作最適化チャレンジ"のためのチュートリアルです.   
このチュートリアルではシミュレータの動きを確認し, 制御アルゴリズムを実装してその精度を確認するまでの一連の流れを説明します.

## 目次
1. 前分析
1. クレーンシミュレータを制御するためのアルゴリズム
1. クレーンシミュレータの制御
1. 提出用のファイルの作成
1. さらなる分析

## 1. 前分析
連続したレバー値を入力することでクレーンシミュレータがどのように動作するかを確認します.

### レバー値のシーケンスの作成
レバー値は"float"で0.0~1.0までの値をとることに注意してください.

In [ ]:
sequence_1 = []
for i in range(1001):
    if i <= 100:
        sequence_1.append(0.01*i)
    elif i > 100 and i <= 200:
        sequence_1.append(2-0.01*i)
    else:
        sequence_1.append(0.0)

In [ ]:
sequence_2 = []
for i in range(1001):
    if i <= 500:
        sequence_2.append(0.002*i)
    elif i > 500 and i <= 1000:
        sequence_2.append(2-0.002*i)

In [ ]:
sequence_3 = []
for i in range(1001):
    if i <= 500:
        sequence_3.append(0.5)
    else:
        sequence_3.append(0.0)

In [ ]:
sequence_4 = []
for i in range(1001):
    sequence_4.append(0.5)

### シーケンスの可視化
横軸を時間としてシーケンスを可視化します.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
t = np.arange(1001)*0.01

In [ ]:
fig, axes = plt.subplots(2, 2,sharex=True, sharey=True, figsize=(15,10))
axes[0,0].plot(t, sequence_1)
axes[0,0].set_title('sequence_1', fontsize=20)
axes[0,0].set_xlabel('time[s]', fontsize=15)
axes[1,0].plot(t, sequence_2)
axes[1,0].set_title('sequence_2', fontsize=20)
axes[1,0].set_xlabel('time[s]', fontsize=15)
axes[0,1].plot(t, sequence_3)
axes[0,1].set_title('sequence_3', fontsize=20)
axes[0,1].set_xlabel('time[s]', fontsize=15)
axes[1,1].plot(t, sequence_4)
axes[1,1].set_title('sequence_4', fontsize=20)
axes[1,1].set_xlabel('time[s]', fontsize=15)
plt.show()

### クレーンシミュレータへのシーケンスの入力
クレーンシミュレータに作成したシーケンスを入力してその出力を見ます.

まずはシミュレータのための設定ファイルを読み込みます.

In [ ]:
import os
import json

# read the parameters
config_dir = './configs'
with open(os.path.join(config_dir, 'env_params.json')) as f:
    env_params = json.load(f)
with open(os.path.join(config_dir, 'sim_params.json')) as f:
    sim_params = json.load(f)

In [ ]:
for k, v in sim_params.items():
    print(k)
    print(v)

In [ ]:
def play(sequence, crane_env):
    # control result
    result = {}
    result['senkai_radian'] = []
    result['circular_displacement'] = []
    result['radial_displacement'] = []

    # get the initial observation
    observation = crane_env.reset()

    # start the simulator
    crane_env.start()
    done = False
    t = 0
    while not done:
        # step foward
        observation, reward, done, info = crane_env.step(sequence[t])
        result['senkai_radian'].append(np.deg2rad(observation['turning_encoder_angle_deg']))
        result['circular_displacement'].append(observation['hook_diff_c_m'])
        result['radial_displacement'].append(observation['hook_diff_r_m'])
        
        t += 1

        if t == len(sequence):
            done = True

    # stop the simulator
    crane_env.stop()
    
    return result

次にシミュレータをインスタンス化します.

In [ ]:
import env_user

crane_env = env_user.CraneEnv(env_params, sim_params)

シミュレータを走らせます.

In [ ]:
result_1 = play(sequence_1, crane_env)
result_2 = play(sequence_2, crane_env)
result_3 = play(sequence_3, crane_env)
result_4 = play(sequence_4, crane_env)

### 出力の可視化
得られた出力を可視化します.  
角速度が観測不可のため, 'senkai_radian'('turning_encoder_angle_deg')により計算します.

In [ ]:
def visualize(result, sequence):
    # computing the angular velocity
    senkai_velocity = []
    for i in range(len(result['senkai_radian'])):
        if i == 0:
            senkai_velocity.append(0)
        else:
            senkai_velocity.append((result['senkai_radian'][i]-result['senkai_radian'][i-1])/0.01)
    result['senkai_velocity'] = senkai_velocity

    # Plotting the result
    plt.figure(figsize=(12,6))
    plt.plot(np.arange(len(result['circular_displacement']))*0.01, result['circular_displacement'], label='circular displacement [m]')
    plt.plot(np.arange(len(result['radial_displacement']))*0.01, result['radial_displacement'], label='radial displacement [m]')
    plt.plot(np.arange(len(result['senkai_radian']))*0.01, result['senkai_radian'], label="senkai angle [rad]")
    plt.plot(np.arange(len(result['senkai_velocity']))*0.01, result['senkai_velocity'], label="senkai velocity [rad/s]")
    plt.plot(np.arange(len(sequence))*0.01, sequence, label="lever sequence [-1 ~ 1]")

    plt.xlabel('Time [s]', fontsize=15)
    plt.legend()
    plt.show()
    plt.close()

In [ ]:
visualize(result_1, sequence_1)

In [ ]:
visualize(result_2, sequence_2)

In [ ]:
visualize(result_3, sequence_3)

In [ ]:
visualize(result_4, sequence_4)

円周方向の変位(circular displacement)と動径方向の変位(radial displacement)が相対的にレバー値に対して敏感に変化することが分かります. よってレバー値は徐々に増加させていく必要がありそうです.

## 2. クレーンシミュレータを制御するためのアルゴリズム
上記の前分析の結果をもとに, シンプルなルールベースの制御アルゴリズムを考案し, 実装します.

### Agentの実装
クレーンシミュレータを制御するためのagentを実装します.  
agentは以下のメソッドを持つ必要があります:  

- `get_model`
- `set_params`
- `policy`

`get_model`メソッドはagentをインスタンス化し, もし使われていたならばモデルを読み込みます. `set_params`メソッドは'senkai_target_angle', 'weight_t', 'wire_ratio'等のパラメータを読み込みます. `policy`メソッドは観測値が渡されてから次のレバー値を返します. レバー値は, 0~1を満たしている必要があります.

`set_params`メソッドに渡されるパラメータは以下です.

|  名前  |  データ型  |  単位  |  範囲  |  デフォルト  |  説明  |
| ---- | :----: | :----: | :----: | :----: | ---- |
|  senkai_target_angle  |  float  |  deg  |  0 ~ 180  |  -  |  目標とする旋回角度  |
|  weight_t  |  float  |  t  |  0 ~ 4  |  1  |  吊り荷重さ(フックは除く)  |
|  wire_ratio  |  float  |  -  |  0.1 ~ 0.9  |  0.8  |  ブームトップから地面までの距離の割合  |
|  init_kifuku_deg  |  float  |  deg  |  10 ~ 80  |  60  |  起伏角度 初期値  |
|  init_yure_senkai_m  |  float  |  m  |  -1 ~ 1  |  0  |  円周方向の荷揺れ 初期値  |
|  init_yure_kifuku_m  |  float  |  m  |  -1 ~ 1  |  0  |  半径方向の荷揺れ 初期値  |
|  param_randomize_seed  |  uint  |  -  |  -  |  0  |  内部パラメータのランダマイズ設定(== 0 : seed is not fixed)  |

`policy`メソッドに渡される観測値は以下です.

|  名前  |  データ型  |  単位  |  説明  |
| ---- | :----: | :----: | ---- |
|  step  |  uint  |  -  |  シミュレーションの実行ステップ数  |
|  turning_lever_value  |  float  |  -  |  旋回レバー操作量  |
|  turning_encoder_angle_deg  |  float  |  deg  |  旋回エンコーダ角度  |
|  turning_encoder_acquisition_time  |  float  |  ms  |  旋回エンコーダ取得時刻  |
|  working_radius_m  |  float  |  m  |  作業半径  |
|  actual_load_t  |  float  |  t  |  実荷重  |
|  main_wire_length_m  |  float  |  m  |  主巻吊荷長さ  |
|  boom_top_x_m  |  float  |  m  |  ブームトップのx座標  |
|  boom_top_y_m  |  float  |  m  |  ブームトップのy座標  |
|  boom_top_z_m  |  float  |  m  |  ブームトップのz座標  |
|  boom_top_status  |  uint  |  -  |  座標更新時 = 1, 非更新時 = 0. 値は10Hzごとに更新される.  |
|  left_turning_pressure_mpa  |  float  |  MPa  |  左旋回圧力  |
|  right_turning_pressure_mpa  |  float  |  MPa  |  右旋回圧力  |
|  tawami_diff_deg  |  float  |  deg  |  円周方向のたわみ量  |
|  hook_x_m  |  float  |  m  |  フック先端のx座標  |
|  hook_y_m  |  float  |  m  |  フック先端のy座標  |
|  hook_z_m  |  float  |  m  |  フック先端のz座標  |
|  hook_status  |  uint  |  -  |  座標更新時 = 1, 非更新時 = 0. 値は10Hzごとに更新される.  |
|  hook_diff_c_m  |  float  |  m  |  円周方向の荷揺れ量  |
|  hook_diff_r_m  |  float  |  m  |  半径方向の荷揺れ量  |

シミュレーションが走るとまずシミュレータがインスタンス化されます. 続いて`get_model`メソッドが呼ばれてagentがインスタンス化され, `set_params`メソッドによってクレーンのパラメータや外的なパラメータが読み込まれます. 各ステップにおいて, シミュレータから観測値が生成され, `policy`メソッドにそれが渡されて次の入力レバー値が返されます. そのレバー値がシミュレータに渡されてまた新たな観測値が生成されます. この一連のやり取りが`done`フラグが`True`になるまで繰り返し実行されます. エピソードは, 入力レバー値が1000ステップの間0になるか合計ステップ数が100000以上になると終了します(`done`フラグが`True`になる).

```
# instanciate the simulator
crane_env = env_user.CraneEnv(env_params, sim_params)

# instanciate the agent
crane_agent = agent.Agent.get_model('./model')
crane_agent.set_params(external_params)

# get the initial observation
observation = env.reset()

# run the simulation
done = False
while not done:
    # next lever value
    action = crane_agent.policy(observation)

    # step foward
    observation, reward, done, info = crane_env.step(action)
```
単純なルールベースアルゴリズムを実装するために, いくつかのモデルパラメータを`get_model`メソッド内に定義します.  
モデルパラメータは以下です:

- max_steps
- max_velocity
- del_lever
- max_angle

どのように動作するかに関しては`policy`メソッドを確認してください.    
角速度が観測不可のため, 観測可能な'turning_encoder_angle_deg'によって角速度を計算します.

In [ ]:
class Agent(object):
    def __init__(self, action_range):
        self.model = None
        self.action_range = action_range
        self.step = 0
        self.observations = self.init_log()

    @classmethod
    def get_model(cls, model_path):
        action_range = (0.0, 1.0)
        agent = cls(action_range)

        # load some model
        if os.path.exists(model_path):
            with open(model_path) as f:
                agent.model = json.load(f)
        else:
            agent.model = {'max_steps': 5000,
                           'max_velocity': 0.3,
                           'del_lever': 0.01,
                           'max_angle': 0.8}

        return agent

    def set_params(self, params):
        """
        args:
          params (data type: dict)
            - 'senkai_target_angle'
            - 'weight_t'
            - 'wire_ratio'
            - 'init_kifuku_deg'
            - 'init_yure_senkai_m'
            - 'init_yure_kifuku_m'
            - 'param_randomize_seed'
        """
        self.params = params

    def policy(self, observation):
        """
        args:
          observation (data type: dict):
            - 'step'
            - 'turning_lever_value'
            - 'turning_encoder_angle_deg'
            - 'turning_encoder_acquisition_time'
            - 'working_radius_m'
            - 'actual_load_t'
            - 'main_wire_length_m'
            - 'boom_top_x_m'
            - 'boom_top_y_m'
            - 'boom_top_z_m'
            - 'boom_top_status'
            - 'left_turning_pressure_mpa'
            - 'right_turning_pressure_mpa'
            - 'tawami_diff_deg'
            - 'hook_x_m'
            - 'hook_y_m'
            - 'hook_z_m'
            - 'hook_status'
            - 'hook_diff_c_m'
            - 'hook_diff_r_m'
        available observation

        returns:
          next_lever_value(data type: float, -1 <= and <= 1)
        """
        # save the observations
        if self.step > observation['step']:
            self.observations = self.init_log()
            self.step = 0
        self.observations['senkai_radian'].append(np.deg2rad(observation['turning_encoder_angle_deg']))

        # computing the angular velocity
        velocity = 0.0
        if len(self.observations['senkai_radian']) > 1:
            velocity = (self.observations['senkai_radian'][-1]-self.observations['senkai_radian'][-2])/0.01
        self.observations['senkai_velocity'].append(velocity)

        # computing the next lever value
        next_lever_value = 0.0
        if self.step <= self.model['max_steps']:
            if observation['turning_encoder_angle_deg'] < self.params['senkai_target_angle']*self.model['max_angle']:
                if self.observations['senkai_velocity'][-1] <= self.model['max_velocity']:
                    next_lever_value = observation['turning_lever_value'] + self.model['del_lever']
                else:
                    next_lever_value = observation['turning_lever_value'] - self.model['del_lever']

            else:
                if self.observations['senkai_velocity'][-1] <= 0.0:
                    next_lever_value = observation['turning_lever_value'] + self.model['del_lever']
                else:
                    next_lever_value = observation['turning_lever_value'] - self.model['del_lever']

        next_lever_value = min(max(next_lever_value, self.action_range[0]), self.action_range[1])

        self.step += 1

        return next_lever_value

    def init_log(self):
        observations = {}
        observations['senkai_radian'] = []
        observations['senkai_velocity'] = []

        return observations

### モデルパラメータの最適化
ベイズ最適化( https://github.com/fmfn/BayesianOptimization )を用いてモデルパラメータを最適化します.  
最適化するパラメータは以下です:

- max_steps
- max_velocity
- del_lever
- max_angle

目的変数はコンペティションで定義されているスコア( https://signate.jp/competitions/428#evaluation )です. これは"evaluate.py"で実装されている関数で計算が可能です.

In [ ]:
import time
from bayes_opt import BayesianOptimization, UtilityFunction
from evaluate import evaluation_crane

class Optimizer(object):
    def __init__(self, env, agent, evaluation_params, external_params):
        self.env = env
        self.agent = agent
        self.evaluation_params = evaluation_params
        self.external_params = external_params

    def play(self):
        # get the initial observation
        observation = self.env.reset()

        # start the simulator
        self.env.start()
        done = False
        while not done:
            # time the control
            time_start = time.perf_counter()
            action = self.agent.policy(observation)
            runtime = time.perf_counter() - time_start
            self.env.update_runtime_results(runtime)

            # step foward
            observation, reward, done, info = self.env.step(action)

        # get the control results
        result = self.env.get_control_results()

        # stop the simulator
        self.env.stop()

        return result

    def compute_target(self, result):
        flag, score = evaluation_crane(result, self.evaluation_params, self.external_params)
        if flag:
            print('Score: {}\n'.format(score))
            return score
        else:
            print('Score: {}\n'.format(0.0))
            return 0.0

    def optimize(self, pbounds, n = 10):
        # set the optimizer
        opt = BayesianOptimization(f = None,
                                   pbounds = pbounds,
                                   verbose = 2,
                                   random_state = 1)
        opt.set_gp_params(normalize_y = False)
        utility = UtilityFunction(kind="ucb", kappa=2.5, xi=0.0)

        # run the optimization
        for i in range(n):
            print('Optimization {}:\n'.format(i+1))
            # suggest the next parameters
            next_point = opt.suggest(utility)

            # set the suggested parameters
            self.agent.model = next_point

            # get the result
            result = self.play()

            # compute the score
            score = self.compute_target(result)

            # register the result for the given parmaeters
            opt.register(params = next_point, target = score)

        # set the best model parameters
        self.agent.model = opt.max['params']

agentに渡されるシミュレータの設定や外的パラメータは以下です.

In [ ]:
with open(os.path.join(config_dir, 'env_params.json')) as f:
    env_params = json.load(f)
with open(os.path.join(config_dir, 'sim_params.json')) as f:
    sim_params = json.load(f)
with open(os.path.join(config_dir, 'external_params.json')) as f:
    external_params = json.load(f)
external_params.update(sim_params['state_init'])

with open(os.path.join(config_dir, 'evaluation_params.json')) as f:
    evaluation_params = json.load(f)

print('configurations:')
for k, v in sim_params.items():
    print(' ', k)
    print(' ', v)
print('\nexternal parameters:')
for k, v in external_params.items():
    print(' ', k, v)

モデルパラメータの探索範囲は`pbounds`で設定し, 最適化の繰り返し回数は`n`で設定します.  
そして`optimize`メソッドによって最適化を実行します.

In [ ]:
# instanciate the agent
os.makedirs('model', exist_ok=True)
model_path = os.path.join('model', 'params.json')
action_range = (0.0, 1.0)
agent = Agent(action_range)
agent.set_params(external_params)

# instanciate the optimizer(pass "crane_env" instanciated above which were set "env_params" and "sim_params".)
optimizer = Optimizer(crane_env, agent, evaluation_params, external_params)
pbounds = {'max_steps': (9000, 12000),
           'max_velocity': (0.2, 0.4),
           'del_lever': (0.0005, 0.001),
           'max_angle': (0.4, 0.9)}

# run the optimizer
n = 100
optimizer.optimize(pbounds, n)

# save the best parameters
with open(model_path, 'w', encoding='utf-8') as f:
    json.dump(agent.model, f)

## 3. クレーンシミュレータの制御
最適化されたモデルによってクレーンシミュレータを制御し, その精度を評価します.

### シミュレーションの実行
シミュレーションを実行し, 結果を保存します.

In [ ]:
def run(crane_env, crane_agent, output_path):
    # get the initial observation
    observation = crane_env.reset()
    
    # start the simulator
    crane_env.start()
    done = False
    while not done:
        # time the control
        time_start = time.perf_counter()
        action = crane_agent.policy(observation)
        runtime = time.perf_counter() - time_start
        crane_env.update_runtime_results(runtime)
        
        # step foward
        observation, reward, done, info = crane_env.step(action)

    # save the control results
    crane_env.save_control_results(output_path)

    # stop the simulator
    crane_env.stop()

以下の`output_path`に結果が保存されます.

In [ ]:
crane_agent = Agent.get_model(model_path)
crane_agent.set_params(external_params)
output_path = os.path.join('output', 'control_results.json')
run(crane_env, crane_agent, output_path)

### 結果の評価
結果を評価します.

agentに渡されるシミュレータの設定や外的パラメータは以下です.

In [ ]:
with open(os.path.join(config_dir, 'env_params.json')) as f:
    env_params = json.load(f)
with open(os.path.join(config_dir, 'sim_params.json')) as f:
    sim_params = json.load(f)
with open(os.path.join(config_dir, 'external_params.json')) as f:
    external_params = json.load(f)
external_params.update(sim_params['state_init'])
print('configurations:')
for k, v in sim_params.items():
    print(' ', k)
    print(' ', v)
print('\nexternal parameters:')
for k, v in external_params.items():
    print(' ', k, v)

上記の条件のもとで評価結果を確認します.

In [ ]:
# read parameters
with open(os.path.join(config_dir, 'evaluation_params.json')) as f:
    params = json.load(f)
with open(os.path.join(config_dir, 'external_params.json')) as f:
    external_params = json.load(f)
print('Parameters for evaluation:')
for k, v in params.items():
    print(' ', k, v)
for k, v in external_params.items():
    print(' ', k, v)

# read the control result
with open('./output/control_results.json') as f:
    result = json.load(f)
print('\nCategories for evaluation:')
for k in result.keys():
    print(' ', k)

# evaluate the result
print('\nEvaluation results:')
flag, score = evaluation_crane(result, params, external_params)

## 4. 提出用のファイルの作成
上記のアルゴリズムを実装した応募用ファイルを作成します.

### "agent.py"の編集

提出用のファイルを作成するために上記で定義した`Agent`クラスを"agent.py"で作成します("README"も参照).

### シミュレータに対する制御

以下のコマンドを実行して期待通りの制御を行うか確認します("README"も参照).

In [ ]:
!python run.py

### 制御アルゴリズムの評価

以下のコマンドを実行して期待通りの精度となるか確認します("README"も参照).

In [ ]:
!python evaluate.py

### zipファイルの作成

そして以下のコマンドを実行すれば提出用のzipファイルが作成されます("README"も参照).

In [ ]:
!bash submit.sh

## 5. さらなる分析

このチュートリアルでは単純なルールベースのアルゴリズムを実装し, そのパラメータを最適化しました.  
限られた条件しか考慮しなかったため, よりアルゴリズムを頑健にするために改良しなければなりません(例えばさらにモデルパラメータを追加するなど).

強化学習アルゴリズムがこの問題に対処できる可能性があります(またはルールベースアルゴリズムと組み合わせることでさらに良くすることも考えられます). 強化学習アルゴリズムを利用するためには各ステップにおいて環境から生成される"報酬"を定義しなければなりませんが, このコンペティションでは"報酬"は定義されていません. 高い精度を達成するように"報酬"を適切に考案することが強化学習アルゴリズムを利用する上で重要になりそうです. "報酬"は"env_user.py"の`step`メソッドで実装できます(デフォルトでは常に0を返します).

素晴らしいアイディアを楽しみにしています.  
健闘を祈ります!